In [13]:
# 필요한 라이브러리 설치
# pip install pandas folium geopandas

import pandas as pd
import folium

# 1. 보행등 데이터 불러오기
gwangjin_lights = pd.read_csv('/Users/heejin/code/gongmo/src/data/stat_xy.csv', encoding='euc-kr')
gwangjin_lights = gwangjin_lights[gwangjin_lights['자치구'] == '광진구']

# 2. 노인 인구 데이터 불러오기
population = pd.read_csv('/Users/heejin/code/gongmo/src/data/oldman.csv', encoding='euc-kr')

# 필요한 컬럼만 선택
population = population[['행정구역', '2025년03월_65세이상전체']]
# 숫자형 변환 (쉼표 제거)
population['2025년03월_65세이상전체'] = population['2025년03월_65세이상전체'].str.replace(',', '').astype(int)

# 3. 광진구 지도 만들기
gwangjin_center = [37.5384, 127.0823]  # 광진구 중심
m = folium.Map(location=gwangjin_center, zoom_start=13)

# 보행등을 파란 점으로 표시
for idx, row in gwangjin_lights.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# 동별 대략 중심 좌표 (임의 설정)
dummy_locations = {
    '화양동': [37.5422, 127.0717],
    '군자동': [37.5555, 127.0735],
    '중곡제1동': [37.5647, 127.0771],
    '중곡제2동': [37.5640, 127.0867],
    '중곡제3동': [37.5652, 127.0846],
    '중곡제4동': [37.5681, 127.0867],
    '능동': [37.5487, 127.0782],
    '광장동': [37.5462, 127.1048],
    '자양제1동': [37.5316, 127.0812],
    '자양제2동': [37.5311, 127.0906],
    '자양제3동': [37.5263, 127.0878],
    '자양제4동': [37.5294, 127.0719],
    '구의제1동': [37.5377, 127.0867],
    '구의제2동': [37.5384, 127.0834],
    '구의제3동': [37.5400, 127.0920],
}

# 색상 결정 함수 (노인 인구 수에 따라)
def get_color(elderly_count):
    if elderly_count > 4000:
        return 'darkred'
    elif elderly_count > 3000:
        return 'red'
    elif elderly_count > 2000:
        return 'orange'
    else:
        return 'yellow'

# 동별 노인 인구를 지도에 표시
for idx, row in population.iterrows():
    dong_name = None
    for key in dummy_locations.keys():
        if key in row['행정구역']:
            dong_name = key
            break
    if dong_name:
        lat, lon = dummy_locations[dong_name]
        folium.Circle(
            location=[lat, lon],
            radius=300,
            color=get_color(row['2025년03월_65세이상전체']),
            fill=True,
            fill_opacity=0.4
        ).add_to(m)

# 4. 지도 저장
m

m.save('/Users/heejin/Downloads/gwangjin_map.html')